# 🛡️ The Swordsmith's Quest — Young's Modulus Detective

You are an apprentice swordsmith on a mission to choose the best metal to forge a legendary sword.
Use **linear regression** on stress–strain data to estimate **Young's modulus** and compare to reference metals.

Follow the levels, earn badges, and become the **Master Swordsmith**! ⚔️

## ⚙️ Setup & Instructions

The data and code in the first part of the jupyter notebook (Young's modulus) is based on that published by Michael N Sakano, Saaketh Desai and Alejandro Strachan from Purdue University on [nanoHUB](https://nanohub.org/resources/youngsmod)<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1) under the GNU General Public License version 3. It was adapted by Berit Zeller-Plumhoff and Bianca Constante Guedert for teaching at the University fo Rostock. The modifications include shortening of the jupyter notebook, adding comments, gamifying and rephrasing text passages.

<a name="cite_note-1"></a>1.[^](#cite_ref-1) Michael N Sakano, Saaketh Desai, Alejandro Strachan (2020), "Linear Regression Young's modulus," https://nanohub.org/resources/youngsmod. (DOI: 10.21981/3NK6-A772).


- First it download the dataset from the original notebook URL.
- If the download is forbidden, it will fall back to a synthetic example so the exercise remains runnable.


Run cells in order. Use the slider to select the elastic region for regression.

## Learning objectives

After going over this notebook you will be able to:

- Use Pandas data structures to organize your data
- Use linear regression to obtain Young’s modulus and yield stress from stress-strain data
- Explore how the fit parameters affect the results

## Libraries

We begin by loading the libraries required to perform the requires tasks. These include:

- [Pandas](https://pandas.pydata.org/docs/) to load and organize the data
- [Scikit-learn](https://scikit-learn.org/stable/index.html) to apply the linear regression model
- [Matplotlib](https://matplotlib.org/) and [plotly](https://plotly.com/) for plotting the data

In [ ]:
import pandas as pd # This library is for developing data structures in the form of tables
import numpy as np # This library is for scientific operations and data manipulation in matrices
from sklearn import datasets, linear_model # This library helps to develop the linear model
from sklearn.metrics import mean_squared_error # This library adds error metrics to our model

import matplotlib.pyplot as plt # This library is for visualizing the curves
import plotly.express as px #This library is for visualizing advanced graphics
import plotly.graph_objs as go # This library is the graphical object for plotly


## Level 1 — The Forge's Ledger (Load & Inspect Data) — 10 pts

**Mission:** Load the stress–strain data from Hollomon, J. H. Tensile Stress-Strain Curves of a 70-30 Brass (1944) — the original nanohub notebook dataset — and display the first rows and column names.

**Tasks**: 
- [  ] Load Dataset
- [  ] Display the first rows and columns names

**Badge**: *Supply Guardian* 🏅

<details>
<summary>💡 Hint</summary>
Look for two columns: strain (or epsilon) and stress (or sigma). Use `pandas.read_csv` to download a CSV; 

In [ ]:
score = 0
section_scores = {
    "section_1" : 0, #max 10pts
    "section_2" : 0, #max 30pts
    "section_3" : 0, #max 60pts
}

In [ ]:
# TASK: define the path. Where is your data?
# Set the name of the data file, which should be organized in two columns,
# Strain in column 1 and stress in column 2.
# The .csv file should be in the same folder as the jupyter notebook otherwise, you need to adjust the filename

### BEGIN SOLUTION
path = './data/Brass_grainsize_0_015mm.csv'

# load your data; keep it simple with pandas
data = pd.read_csv(path, delimiter=',')

# display the "head" (first five rows) of the dataframe
# try different commands of displaying the dataframe, 

data.tail()
### END SOLUTION

In [ ]:
# HIDDEN TEST CELL: Level 1
import pandas as pd

max_points = 10  # maximum points for this section

try:
    assert 'data' in globals(), "Variable 'data' not defined."
    assert isinstance(data, pd.DataFrame), "'data' must be a pandas DataFrame."
    assert 'Strain' in data.columns, "'data' must contain 'Strain' column."
    assert 'Stress (MPa)' in data.columns, "'data' must contain 'Stress (MPa)' column."
    
    # Add points only up to max
    added = max_points - section_scores["section_1"]
    if added > 0:
        score += added
        section_scores["section_1"] = max_points

    print(f"✅ Level 1 passed: +{added} pts (max {max_points} pts)")

except AssertionError as e:
    print(f"❌ Level 1 failed: {e}")





## Level 2 — The Plotting Rite (Visualize & Select Elastic Region) — 20 pts

**Mission:** Plot stress vs strain and use the slider to select an elastic-region window for regression. Assign the dataframe columns to variables `strain` and `stress`, and convert `strain` to percentage. Verify by printing the first 5 values of each variable.  

**Badge:** *Eyes of the Forger* 🏅

**Tasks:**  
- [ ] Assign dataframe columns to `strain` and `stress`  
- [ ] Convert `strain` values to %  
- [ ] Print the first 5 values of both variables  
- [ ] Plot stress vs strain  

<details>
<summary>💡 Hint</summary>
You can assign columns with `strain = data['strain']` and `stress = data['stress']`. Multiply `strain` by 100 to get percentages. Use `matplotlib.pyplot` for plotting.
</details>

In [ ]:
#STUDENT TASK

### BEGIN SOLUTION
strain = data['Strain']*100
stress = data['Stress (MPa)']

print('Strain (%) \n', strain[:5])
print('Stress (MPa) \n', stress[:5])
### END SOLUTION


Plotting

Since we have now loaded the data, you can visualize it using matplotlib or plotly. The advantage of plotly is the interactiveness of the resulting plot.

Display the stress-strain curve of the loaded data, including axes labels and a figure title

In [ ]:
#PLOT
### BEGIN SOLUTION
fig = px.line(x=strain, y=stress, title="Stress-Strain") 
fig.update_layout(xaxis_title='Strain (%)',
                   yaxis_title='Stress (MPa)')
fig.show()
### END SOLUTION

Based on the data, decide on the maximum strain defining the linear regime of deformation. Based on this region, you will fit a linear curve to the data to determine the Young's modulus of the material. 
Automatically find the final array index of the strain variable pertaining to this maximum strain using the [numpy where](https://numpy.org/doc/stable/reference/generated/numpy.where.html) function.

In [ ]:
# insert the maximum strain you wish to include in the fitting for the
# Young's modulus based on the graph above
### BEGIN SOLUTION
max_strain = 0.05

# us the numpy where function to define the maximum index of the variable
# to include
max_index = np.min(np.where(strain>max_strain))
print('Max index', max_index)
### END SOLUTION

In [ ]:
# HIDDEN TEST CELL: Section 2 - max_index
import numpy as np

max_points = 30  # maximum points for this section

try:
    # --- Variable existence ---
    for var in ['max_strain', 'max_index']:
        assert var in globals(), f"Variable '{var}' is not defined."

    # --- Type checks ---
    assert isinstance(max_index, (int, np.integer)), "'max_index' must be an integer."
    assert isinstance(max_strain, (float, int)), "'max_strain' must be a number."

    # --- Value check based on your dataset ---
    expected_max_index = max_index
    expected_max_strain = 0.05

    assert np.isclose(max_strain, expected_max_strain), f"max_strain should be {expected_max_strain}, got {max_strain}"
    assert max_index == expected_max_index, f"max_index should be {expected_max_index}, got {max_index}"

    # --- Add points only if not already awarded ---
    added = max_points - section_scores.get("max_index", 0)
    if added > 0:
        score += added
        section_scores["max_index"] = max_points

    print(f"✅ Section 2 passed: +20 pts. Total score: {max_points} (max {max_points} pts)")

except AssertionError as e:
    print(f"❌ Section 2 failed: {e}")




## Level 3 — The Forge Equation (Linear Regression to obtain Young's Modulus) — 30 pts

**Mission:** Using the selected region, fit a linear model (stress = E * strain) and report the estimated **E_modulus** in Pascals.

Badge: *Keeper of Elastic Wisdom* 🏅

In [ ]:
# STUDENT TASK: Run regression on the selected region and set variable `E_modulus` (float, Pa)
### BEGIN SOLUTION
x_train=np.array(strain[:max_index]).reshape(-1,1)
y_train=np.array(stress[:max_index]).reshape(-1,1)
### END SOLUTION



Following the definition of your training data, you will now write a function that uses that data to train a linear regression model based on the scikit-learn library. The function should take your training data as input and output the parameters of the fitted model, as well as the fitted stress values.

In [ ]:
#CREATING THE LINEAR MODEL
### BEGIN SOLUTION
def regression(X_train, Y_train):
    # Define the model using linear_model and LinearRegression from Scikit_learn
    model = linear_model.LinearRegression()
    # train the model using .fit
    model.fit(X_train, Y_train)
        
    # Use the model to predict the entire set of data using .predict
    predictions = model.predict(X_train) 
    
    # return the predictions and the fitted model
    return predictions, model
### END SOLUTION



Apply the function you have just defined to your data. Based on the predictions, determine the mean squared error of the predictions and print the linear function as well as the error.

In [ ]:
### BEGIN SOLUTION
predictions, model = regression(x_train, y_train) # This line calls the Regression model implemented in the function 

# Print model and mean squared error and variance score
print("Linear Equation: %.4e X + (%.4e)"%(model.coef_, model.intercept_))
print("Mean squared error: %.4e" % (mean_squared_error(y_train, predictions)))
### END SOLUTION



Plot the original experimental data, the training data and the linear model predictions in one plot with different colours, adding a legend.

In [ ]:
layout0= go.Layout(hovermode= 'closest', width = 800, height=600, showlegend=True,  
                       # Hovermode establishes the way the labels that appear when you hover are arranged 
                       # Establishing a square plot width=height
xaxis= dict(title=go.layout.xaxis.Title(text="Axial Strain (%)", font=dict(size=24)), zeroline= False, 
                gridwidth= 1, tickfont=dict(size=18)), # Axis Titles. Removing the X-axis Mark. Adding a Grid
yaxis= dict(title=go.layout.yaxis.Title(text="Stress (MPa)", font=dict(size=24)), zeroline= False, 
                gridwidth= 1, tickfont=dict(size=18)), # Axis Titles. Removing the Y-axis Mark. Adding a Grid
legend=dict(font=dict(size=24))) # Adding a legend
    
training = go.Scatter(x = strain[:max_index], y = stress[:max_index], mode = 'markers', 
                          marker= dict(size= 10, color= 'red'), name= "Training Data") 

prediction = go.Scatter(x = strain[:max_index], y = predictions.reshape(1,-1).tolist()[0], mode = 'lines', 
                            line = dict(color = "green", width = 1.5),name= "Model") 
    
full = go.Scatter(x = strain, y = stress, mode = 'markers',
                      marker= dict(size= 5, color= 'black'), name= "Full Data")

data = [full, training, prediction]
fig= go.Figure(data, layout=layout0)

fig.show()

Calculate and print the Young's modulus of the material in GPa, and cross-check your computed value with the published literature.

Now repeat the above with different thresholds for the maximum strain and evaluate how your linear model and Young's modulus change with different values. Repeat the workflow for the additional Brass_grainsize_0_020mm.csv and Brass_grainsize_0_025mm.csv files to assess how the Young's modulus changes with grain size - these are 0.015, 0.02 and 0.025 mm, respectively.

In [ ]:
#you can simply run the whole script until (incl.) cell for the three datasets and note down the E_mod value. 
#Alternatively, you could write a function that incorporates all these cells and automatically assess the changing E_mod
#in this way. We will have a look at this in the future.
### BEGIN SOLUTION

E_mod=model.coef_/1000*100 # convert to GPa
print("Young's modulus (GPa)",E_mod)
#0.015 mm 170.883205
#0.020 mm 169.0934015
#0.025 mm 167.8463072
gs=[0.015,0.02,0.025]
E=[170.88,169.09,167.85]
fig=px.scatter(x=gs,y=E)
fig.update_layout(xaxis_title='Grain size (mm)', yaxis_title='Young\'s modulus (GPa)')
# the Young's modulus decreases with increasing grain size
# all values are higher than those found online with ranged between 90-140 GPa - these differences may be due to differences in the microstructure, or the accuracy of the experiment, which is almost 80 years old

### END SOLUTION

In [ ]:
# HIDDEN TEST CELL: Regression + E_mod
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

max_points = 60  # max points for this section

try:
    # --- Existence checks ---
    for var in ['x_train', 'y_train', 'regression', 'model', 'predictions', 'E_mod']:
        assert var in globals(), f"Variable '{var}' is not defined."

    # --- Type and regression checks ---
    assert isinstance(x_train, np.ndarray)
    assert isinstance(y_train, np.ndarray)
    assert callable(regression)
    assert hasattr(model, 'coef_')
    assert hasattr(model, 'predict')

    pred_test, mod_test = regression(x_train, y_train)
    assert np.allclose(pred_test, model.predict(x_train), atol=1e-6)

    if isinstance(E_mod, (list, np.ndarray)):
        E_val = float(np.array(E_mod).ravel()[0])
    else:
        E_val = float(E_mod)
    assert 160 <= E_val <= 180, f"E_mod out of range: {E_val:.2f} GPa"

    # Add points only if not already added
    added = max_points - section_scores["section_3"]
    if added > 0:
        score += added
        section_scores["section_3"] = max_points

    print(f"✅ Regression test passed: +30 pts. Total score: {max_points} (max {max_points} pts)")

except AssertionError as e:
    print(f"❌ Regression test failed: {e}")





## Level 4 — The Tournament of Alloys (Compare to Reference Metals) — 20 pts

**Mission:** Compare your estimated modulus to a reference table (Steel, Aluminum, Titanium). Identify which metal your sample is closest to (within 10%).

Badge: *Alloy Recognizer* 🏅

In [ ]:
# Compare your estimated Young's modulus to reference metals (all in GPa)

# Your computed modulus in GPa
if isinstance(E_mod, (list, np.ndarray)):
    E_value = float(np.array(E_mod).ravel()[0])
else:
    E_value = float(E_mod)

# Reference Young's modulus (GPa) TODO: Add references as you wish !
references = {
    'Aluminum': 69,
    'Steel (generic)': 200,
    'Titanium': 116
}

print("Reference Young's modulus (approx.):")
for metal, val in references.items():
    print(f"  {metal}: {val:.2f} GPa")

# Compute relative differences
differences = {metal: abs(E_value - val)/val for metal, val in references.items()}

# Identify closest material
closest = min(differences, key=differences.get)
rel_error = differences[closest]

# Feedback
print("\n🎯 Your estimated Young's modulus:")
print(f"  {E_value:.2f} GPa")

if rel_error <= 0.10:
    print(f"✅ Your sample is closest to {closest} (within 10%, relative error {rel_error:.2%})")
elif rel_error <= 0.25:
    print(f"⚠️ Your sample is somewhat close to {closest} (relative error {rel_error:.2%})")
else:
    print(f"❌ Your sample does not closely match any reference metal (closest: {closest}, relative error {rel_error:.2%})")

# Optional: show ranking of all metals
print("\n📊 Ranking of all reference metals by closeness:")
sorted_mets = sorted(differences.items(), key=lambda x: x[1])
for i, (metal, diff) in enumerate(sorted_mets, 1):
    print(f"  {i}. {metal} - relative error {diff:.2%}")



## Final Boss — The Sword Choice (Identify Best Metal) — 40 pts

**Mission:** Based on mechanical properties (Young's modulus you found) and a short reasoning (3–5 sentences), choose which metal you would use to forge the legendary sword and why.

Badge: *Master Swordsmith* 🏆

Write your decision and reasoning below (3–5 sentences). 

_Example answer:_ "I would choose Steel because the estimated modulus of ~200 GPa indicates high stiffness and the material's known toughness suits a sword's requirements. However, if weight is a concern, Titanium might be preferred despite lower stiffness."

_Write your reflection here:_

In [ ]:
### BEGIN SOLUTION

### END SOLUTION

## 🎉 Quest Complete!

You have completed the Swordsmith's Quest. If you earned enough points, you may be awarded the **Master Swordsmith** title.

Next ideas: add uncertainty estimation (bootstrap), use stress units in MPa in plots, or try different candidate alloys.